#  Import Libraries

In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import warnings

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

# Imbalanced Learning
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (
    AdaBoostClassifier, BaggingClassifier, VotingClassifier, StackingClassifier,
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Metrics
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    f1_score, recall_score, precision_score, roc_auc_score,
    balanced_accuracy_score  # <-- ADD THIS METRIC
)

# Filter warnings for a cleaner output
warnings.filterwarnings("ignore")

# Parameters and Save Path

In [4]:
# --- Parameters and Configuration ---

# Define the main directory for saving all analysis outputs
SAVE_PATH = 'Analysis'
os.makedirs(SAVE_PATH, exist_ok=True)

# Define subdirectories for better organization
os.makedirs(os.path.join(SAVE_PATH, 'plots'), exist_ok=True)
os.makedirs(os.path.join(SAVE_PATH, 'reports'), exist_ok=True)
os.makedirs(os.path.join(SAVE_PATH, 'results'), exist_ok=True)

# Set a random state for reproducibility of results
RANDOM_STATE = 42

print(f"All analysis outputs will be saved to the '{SAVE_PATH}' directory.")

All analysis outputs will be saved to the 'Analysis' directory.


# Dataset Load & Preprocessing

In [5]:
# --- Step 1: Load and Clean Data ---
print("Loading datasets...")
data_cross = pd.read_csv('oasis_cross-sectional.csv')
data_long = pd.read_csv('oasis_longitudinal.csv')

# Standardize column names
data_long = data_long.rename(columns={'EDUC': 'Educ'})
data = pd.concat([data_cross, data_long], ignore_index=True)
data.columns = data.columns.str.strip()

# Fill missing values with the mode
print("Handling missing values...")
for column in data.columns:
    if data[column].isnull().any():
        mode_value = data[column].mode()[0]
        data[column].fillna(mode_value, inplace=True)

# --- Step 2: Binary Classification Setup ---
print("Setting up for binary classification...")
y_binary = data['Group'].replace({'Converted': 'Demented'})
X = data.drop(['Group', 'ID', 'Delay', 'Subject ID', 'MRI ID', 'Visit', 'MR Delay'], axis=1, errors='ignore')

# --- Step 3: Analysis for Research Paper ---
# Outlier Analysis using Boxplots
print("Performing outlier analysis...")
plt.figure(figsize=(15, 10))
sns.boxplot(data=X.select_dtypes(include=np.number))
plt.title('Outlier Analysis via Boxplots')
plt.xticks(rotation=45)
plt.savefig(os.path.join(SAVE_PATH, 'plots', 'outlier_analysis.png'))
plt.close()

# PCA Analysis
print("Performing PCA analysis...")
pca = PCA()
X_scaled_pca = StandardScaler().fit_transform(X.select_dtypes(include=np.number))
pca.fit(X_scaled_pca)
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance Curve')
plt.grid(True)
plt.savefig(os.path.join(SAVE_PATH, 'plots', 'pca_variance_curve.png'))
plt.close()

# --- Step 4: Data Splitting and Encoding ---
print("Splitting and encoding data...")
X_train, X_val, y_train, y_val = train_test_split(
    X, y_binary, test_size=0.2, random_state=RANDOM_STATE, stratify=y_binary
)

label_encoder = LabelEncoder()
y_train_label = label_encoder.fit_transform(y_train)
y_val_label = label_encoder.transform(y_val)

# Define the preprocessing pipeline for features
scale_cols = ['Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']
ohe_cols = ['M/F', 'Hand']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale_cols),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ohe_cols)
    ],
    remainder='passthrough'
)

print("-" * 30)
print("Preprocessing complete.")
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")

Loading datasets...
Handling missing values...
Setting up for binary classification...
Performing outlier analysis...
Performing PCA analysis...
Splitting and encoding data...
------------------------------
Preprocessing complete.
Training set size: 647
Validation set size: 162


# Decision Tree Classifier

In [6]:
# --- Model 1: Decision Tree ---
print("Training Decision Tree...")
pipeline_dt = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', DecisionTreeClassifier(random_state=RANDOM_STATE))
])
params_dt = {'classifier__max_depth': [5, 10, None], 'classifier__min_samples_leaf': [1, 2, 4]}
grid_dt = GridSearchCV(pipeline_dt, params_dt, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training Decision Tree...


# Random Forest Classifier

In [7]:
# --- Model 2: Random Forest ---
print("Training Random Forest...")
pipeline_rf = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', RandomForestClassifier(random_state=RANDOM_STATE))
])
params_rf = {'classifier__n_estimators': [100, 200], 'classifier__max_depth': [10, 20]}
grid_rf = GridSearchCV(pipeline_rf, params_rf, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training Random Forest...


#  Extra Trees Classifier

In [8]:
# --- Model 3: Extra Trees ---
print("Training Extra Trees...")
pipeline_et = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', ExtraTreesClassifier(random_state=RANDOM_STATE))
])
params_et = {'classifier__n_estimators': [100, 200], 'classifier__max_depth': [10, 20]}
grid_et = GridSearchCV(pipeline_et, params_et, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training Extra Trees...


# Gradient Boosting Classifier

In [9]:
# --- Model 4: Gradient Boosting ---
print("Training Gradient Boosting...")
pipeline_gb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', GradientBoostingClassifier(random_state=RANDOM_STATE))
])
params_gb = {'classifier__n_estimators': [100, 200], 'classifier__learning_rate': [0.05, 0.1]}
grid_gb = GridSearchCV(pipeline_gb, params_gb, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training Gradient Boosting...


#  XGBoost Classifier

In [10]:
# --- Model 5: XGBoost ---
print("Training XGBoost...")
pipeline_xgb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', xgb.XGBClassifier(random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss'))
])
params_xgb = {'classifier__n_estimators': [100, 200], 'classifier__learning_rate': [0.05, 0.1], 'classifier__max_depth': [3, 5]}
grid_xgb = GridSearchCV(pipeline_xgb, params_xgb, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training XGBoost...


# LightGBM Classifier

In [11]:
# --- Model 6: LightGBM ---
print("Training LightGBM...")
pipeline_lgbm = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', lgb.LGBMClassifier(random_state=RANDOM_STATE))
])
params_lgbm = {'classifier__n_estimators': [100, 200], 'classifier__learning_rate': [0.05, 0.1]}
grid_lgbm = GridSearchCV(pipeline_lgbm, params_lgbm, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training LightGBM...
[LightGBM] [Info] Number of positive: 401, number of negative: 401
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002498 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1282
[LightGBM] [Info] Number of data points in the train set: 802, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

# CatBoost Classifier

In [12]:
# --- Model 7: CatBoost ---
print("Training CatBoost...")
pipeline_cb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('classifier', cb.CatBoostClassifier(random_state=RANDOM_STATE, verbose=0))
])
params_cb = {'classifier__iterations': [500, 1000], 'classifier__depth': [4, 6]}
grid_cb = GridSearchCV(pipeline_cb, params_cb, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training CatBoost...


# AdaBoost & Bagging Classifiers

In [13]:
# --- Model 8 & 9: AdaBoost and Bagging ---
print("Training AdaBoost...")
pipeline_ada = ImbPipeline([('preprocessor', preprocessor), ('smote', SMOTE(random_state=RANDOM_STATE)), ('classifier', AdaBoostClassifier(random_state=RANDOM_STATE))])
params_ada = {'classifier__n_estimators': [50, 100, 200]}
grid_ada = GridSearchCV(pipeline_ada, params_ada, cv=5, scoring='accuracy').fit(X_train, y_train_label)

print("Training Bagging...")
pipeline_bag = ImbPipeline([('preprocessor', preprocessor), ('smote', SMOTE(random_state=RANDOM_STATE)), ('classifier', BaggingClassifier(random_state=RANDOM_STATE))])
params_bag = {'classifier__n_estimators': [50, 100, 200]}
grid_bag = GridSearchCV(pipeline_bag, params_bag, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training AdaBoost...
Training Bagging...


# MLP & SVM Classifiers

In [14]:
# --- Model 10 & 11: MLP and SVC ---
print("Training MLP Classifier...")
pipeline_mlp = ImbPipeline([('preprocessor', preprocessor), ('smote', SMOTE(random_state=RANDOM_STATE)), ('classifier', MLPClassifier(random_state=RANDOM_STATE, max_iter=1000))])
params_mlp = {'classifier__hidden_layer_sizes': [(50,), (100,)], 'classifier__alpha': [0.0001, 0.001]}
grid_mlp = GridSearchCV(pipeline_mlp, params_mlp, cv=5, scoring='accuracy').fit(X_train, y_train_label)

print("Training SVC...")
pipeline_svc = ImbPipeline([('preprocessor', preprocessor), ('smote', SMOTE(random_state=RANDOM_STATE)), ('classifier', SVC(random_state=RANDOM_STATE, probability=True))])
params_svc = {'classifier__C': [1, 10], 'classifier__kernel': ['rbf', 'linear']}
grid_svc = GridSearchCV(pipeline_svc, params_svc, cv=5, scoring='accuracy').fit(X_train, y_train_label)

Training MLP Classifier...
Training SVC...


# Voting & Stacking Classifiers

In [15]:
# --- Model 12 & 13: Voting and Stacking ---
# Use the best estimators found so far for ensemble methods
clf1 = grid_rf.best_estimator_
clf2 = grid_xgb.best_estimator_
clf3 = grid_cb.best_estimator_

print("Training Voting Classifier...")
voting_clf = VotingClassifier(estimators=[('rf', clf1), ('xgb', clf2), ('cb', clf3)], voting='soft')
voting_clf.fit(X_train, y_train_label)

print("Training Stacking Classifier...")
stacking_clf = StackingClassifier(estimators=[('rf', clf1), ('xgb', clf2), ('cb', clf3)], final_estimator=LogisticRegression(), cv=5)
stacking_clf.fit(X_train, y_train_label)

Training Voting Classifier...
Training Stacking Classifier...


,estimators,"[('rf', ...), ('xgb', ...), ...]"
,final_estimator,LogisticRegression()
,cv,5
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,transformers,"[('scale', ...), ('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None


# Final Comparison and Results

In [ ]:
# --- Final Comparison ---
print("\n" + "="*50)
print("           GENERATING FINAL REPORT")
print("="*50)
all_grids = {
    "Decision Tree": grid_dt, "Random Forest": grid_rf, "Extra Trees": grid_et,
    "Gradient Boosting": grid_gb, "XGBoost": grid_xgb, "LightGBM": grid_lgbm,
    "CatBoost": grid_cb, "AdaBoost": grid_ada, "Bagging": grid_bag,
    "MLP Classifier": grid_mlp, "SVC": grid_svc,
    "Voting Classifier": voting_clf, "Stacking Classifier": stacking_clf
}

results = []
for name, model in all_grids.items():
    print(f"Evaluating {name}...")
    # Make predictions
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else [0]*len(y_val_label)

    # Get metrics
    accuracy = accuracy_score(y_val_label, y_pred)
    # --- NEW METRIC CALCULATION ---
    balanced_accuracy = balanced_accuracy_score(y_val_label, y_pred)
    # --- END NEW METRIC ---
    f1 = f1_score(y_val_label, y_pred)
    recall = recall_score(y_val_label, y_pred)
    precision = precision_score(y_val_label, y_pred)
    roc_auc = roc_auc_score(y_val_label, y_proba)

    # Store results, now including Balanced Accuracy
    results.append({
        "ML Model": name,
        "Accuracy": f"{accuracy*100:.3f}%",
        "Balanced Accuracy": f"{balanced_accuracy*100:.3f}%", # <-- ADDED COLUMN
        "F1 Score": f"{f1*100:.3f}%",
        "Recall": f"{recall*100:.3f}%",
        "Precision": f"{precision*100:.3f}%",
        "ROC_AUC": f"{roc_auc*100:.3f}%"
    })

    # Save confusion matrix and classification report (no changes here)
    report = classification_report(y_val_label, y_pred, target_names=label_encoder.classes_)
    with open(os.path.join(SAVE_PATH, 'reports', f'{name}_report.txt'), 'w') as f:
        f.write(report)

    cm = confusion_matrix(y_val_label, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(f'Confusion Matrix for {name}')
    plt.savefig(os.path.join(SAVE_PATH, 'plots', f'{name}_confusion_matrix.png'))
    plt.close()

# Create and display the final comparison dataframe
# Reorder columns to place Balanced Accuracy next to Accuracy
column_order = ["ML Model", "Accuracy", "Balanced Accuracy", "F1 Score", "Recall", "Precision", "ROC_AUC"]
results_df = pd.DataFrame(results).sort_values(by="Balanced Accuracy", ascending=False).reset_index(drop=True)
results_df = results_df[column_order]

print("\n" + "="*120)
print("TOP CONFIGURATION PER MODEL")
print("="*120)
print(results_df.to_string())

# Save the final results table
results_df.to_csv(os.path.join(SAVE_PATH, 'results', 'final_model_comparison.csv'), index=False)
print(f"\n\nComparison table saved to '{os.path.join(SAVE_PATH, 'results')}'")


           GENERATING FINAL REPORT
Evaluating Decision Tree...
Evaluating Random Forest...
Evaluating Extra Trees...
Evaluating Gradient Boosting...
Evaluating XGBoost...
Evaluating LightGBM...
Evaluating CatBoost...
Evaluating AdaBoost...
Evaluating Bagging...
Evaluating MLP Classifier...
Evaluating SVC...
Evaluating Voting Classifier...
Evaluating Stacking Classifier...

TOP CONFIGURATION PER MODEL
               ML Model Accuracy Balanced Accuracy F1 Score   Recall Precision  ROC_AUC
0         Random Forest  98.765%           99.200%  99.194%  98.400%  100.000%  99.751%
1           Extra Trees  98.765%           99.200%  99.194%  98.400%  100.000%  99.978%
2     Gradient Boosting  98.765%           99.200%  99.194%  98.400%  100.000%  99.416%
3               XGBoost  98.765%           99.200%  99.194%  98.400%  100.000%  99.827%
4              LightGBM  98.765%           99.200%  99.194%  98.400%  100.000%  99.914%
5              CatBoost  98.765%           99.200%  99.194%  98.400

In [19]:
# --- Final Comparison ---
print("\n" + "="*50)
print("           GENERATING FINAL REPORT")
print("="*50)
all_grids = {
    "Decision Tree": grid_dt, "Random Forest": grid_rf, "Extra Trees": grid_et,
    "Gradient Boosting": grid_gb, "XGBoost": grid_xgb, "LightGBM": grid_lgbm,
    "CatBoost": grid_cb, "AdaBoost": grid_ada, "Bagging": grid_bag,
    "MLP Classifier": grid_mlp, "SVC": grid_svc
}

# Add manually trained ensemble models
all_grids["Voting Classifier"] = voting_clf
all_grids["Stacking Classifier"] = stacking_clf

results = []
for name, model in all_grids.items():
    print(f"Evaluating {name}...")
    # Make predictions
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else [0]*len(y_val_label)

    # Get metrics
    accuracy = accuracy_score(y_val_label, y_pred)
    f1 = f1_score(y_val_label, y_pred)
    recall = recall_score(y_val_label, y_pred)
    precision = precision_score(y_val_label, y_pred)
    roc_auc = roc_auc_score(y_val_label, y_proba)

    # Store results
    results.append({
        "ML Model": name,
        "Configuration": "SMOTE + GridSearchCV", # Placeholder for configuration
        "Accuracy": f"{accuracy*100:.3f}%",
        "F1 Score": f"{f1*100:.3f}%",
        "Recall": f"{recall*100:.3f}%",
        "Precision": f"{precision*100:.3f}%",
        "ROC_AUC": f"{roc_auc*100:.3f}%"
    })

    # Save confusion matrix and classification report
    report = classification_report(y_val_label, y_pred, target_names=label_encoder.classes_)
    with open(os.path.join(SAVE_PATH, 'reports', f'{name}_report.txt'), 'w') as f:
        f.write(report)

    cm = confusion_matrix(y_val_label, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(f'Confusion Matrix for {name}')
    plt.savefig(os.path.join(SAVE_PATH, 'plots', f'{name}_confusion_matrix.png'))
    plt.close()

# Create and display the final comparison dataframe
results_df = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)

print("\n" + "="*120)
print("TOP CONFIGURATION PER MODEL")
print("="*120)
print(results_df.to_string())

# Save the final results table
results_df.to_csv(os.path.join(SAVE_PATH, 'results', 'final_model_comparison.csv'), index=False)
print(f"\n\nComparison table saved to '{os.path.join(SAVE_PATH, 'results')}'")


           GENERATING FINAL REPORT
Evaluating Decision Tree...
Evaluating Random Forest...
Evaluating Extra Trees...
Evaluating Gradient Boosting...
Evaluating XGBoost...
Evaluating LightGBM...
Evaluating CatBoost...
Evaluating AdaBoost...
Evaluating Bagging...
Evaluating MLP Classifier...
Evaluating SVC...
Evaluating Voting Classifier...
Evaluating Stacking Classifier...

TOP CONFIGURATION PER MODEL
               ML Model         Configuration Accuracy F1 Score   Recall Precision  ROC_AUC
0         Random Forest  SMOTE + GridSearchCV  98.765%  99.194%  98.400%  100.000%  99.751%
1           Extra Trees  SMOTE + GridSearchCV  98.765%  99.194%  98.400%  100.000%  99.978%
2     Gradient Boosting  SMOTE + GridSearchCV  98.765%  99.194%  98.400%  100.000%  99.416%
3               XGBoost  SMOTE + GridSearchCV  98.765%  99.194%  98.400%  100.000%  99.827%
4              LightGBM  SMOTE + GridSearchCV  98.765%  99.194%  98.400%  100.000%  99.914%
5              CatBoost  SMOTE + GridSearch